In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from IPython.display import display
from pyspark.dbutils import DBUtils
from datetime import date
import os

# Testing reading directory

spark = SparkSession.builder.appName('viadot').getOrCreate()
dbutils = DBUtils(spark)

response = dbutils.fs.ls("/mnt/")
print(response)

22/05/17 12:33:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/17 12:33:00 WARN MetricsSystem: Using default name SparkStatusTracker for source because neither spark.metrics.namespace nor spark.app.id is set.
22/05/17 12:33:03 WARN SparkServiceRPCClient: Cluster 0427-122644-45iadnd in state TERMINATED, waiting for it to start running...
22/05/17 12:33:13 WARN SparkServiceRPCClient: Cluster 0427-122644-45iadnd in state PENDING, waiting for it to start running...
22/05/17 12:33:23 WARN SparkServiceRPCClient: Cluster 0427-122644-45iadnd in state PENDING, waiting for it to start running...
22/05/17 12:33:33 WARN SparkServiceRPCClient: Cluster 0427-122644-45iadnd in state PENDING, waiting for it to s

[FileInfo(path='dbfs:/mnt/DLQ/', name='DLQ/', size=0, modificationTime=0), FileInfo(path='dbfs:/mnt/SPT/', name='SPT/', size=0, modificationTime=0), FileInfo(path='dbfs:/mnt/adls/', name='adls/', size=0, modificationTime=0), FileInfo(path='dbfs:/mnt/azuwevelbwdls01q/', name='azuwevelbwdls01q/', size=0, modificationTime=0), FileInfo(path='dbfs:/mnt/conformed/', name='conformed/', size=0, modificationTime=0), FileInfo(path='dbfs:/mnt/delta/', name='delta/', size=0, modificationTime=1611768799000), FileInfo(path='dbfs:/mnt/raw/', name='raw/', size=0, modificationTime=0)]


In [41]:
from prefect import task, Flow
from prefect.schedules import IntervalSchedule
import pandas as pd
from datetime import timedelta
import datetime

# Flow for writing data every minute to a table

# Convert list to pandas dataframe
@task
def generate_fake_data(data: list):
    return pd.DataFrame(data)

# Convert pandas dataframe to spark dataframe then write to table
@task
def write_to_spark(df: pd.DataFrame):
  sparkdf = spark.createDataFrame(df)
  sparkdf.write.mode("append").saveAsTable("raw.c4c_test4")

# Flow scheduled for every minute
schedule = IntervalSchedule(interval=timedelta(minutes=1))
with Flow("test", schedule=schedule) as f:
    data_raw =[{"Id": "KVSzUaILfQZXDb" + str(datetime.datetime.now()), "AccountId": "EHNYKjSZsiy", "Name": "Turner-Black", "FirstName": "Adam", "LastName": "Carter", "ContactEMail": "Adam.Carter@TurnerBlack.com", "MailingCity": "Jamesport"}]
    df = generate_fake_data(data_raw)
    write_to_spark(df)
    
f.run()

[2022-05-17 13:51:28+0000] INFO - prefect.test | Waiting for next scheduled run at 2022-05-17T13:52:00+00:00
[2022-05-17 13:52:00+0000] INFO - prefect.FlowRunner | Beginning Flow run for 'test'
[2022-05-17 13:52:00+0000] INFO - prefect.TaskRunner | Task 'generate_fake_data': Starting task run...
[2022-05-17 13:52:00+0000] INFO - prefect.TaskRunner | Task 'generate_fake_data': Finished task run for task with final state: 'Success'
[2022-05-17 13:52:00+0000] INFO - prefect.TaskRunner | Task 'write_to_spark': Starting task run...
View job details at https://adb-1930462786844525.5.azuredatabricks.net/?o=1930462786844525#/setting/clusters/0427-122644-45iadnd/sparkUi
[2022-05-17 13:52:02+0000] INFO - prefect.TaskRunner | Task 'write_to_spark': Finished task run for task with final state: 'Success'
[2022-05-17 13:52:02+0000] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
[2022-05-17 13:52:02+0000] INFO - prefect.test | Waiting for next scheduled run at 2022-05-17T

KeyboardInterrupt: 

2022-05-17 13:42:54.358083
